In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
from sklearn import linear_model, svm,model_selection
import sklearn
from arftools import *
import numpy as np
import matplotlib.pyplot as plt
from StringKernel import StringKernel
from cross_validation import validation_croisee

In [2]:
import pandas as pd
import nltk

In [3]:
data = pd.read_json('data/smaller_dataset.json')

In [4]:
def generate_sample(authors):
    authors_sample = authors['reviewerID'].sample(2)
    subset = authors[authors['reviewerID'].isin(authors_sample)]
    X = subset['reviewText']
    Y = subset['reviewerID']
    short_X = []
    for text in X:
        short_X.append(text[0:160])
    return (np.array(short_X),Y)

In [8]:
authors = data[data.groupby(by='reviewerID')['reviewerID'].transform("count") == 10]
samples = [generate_sample(authors)]# for _ in range(3)]<

In [40]:
samples[0][1].shape

(50,)

In [7]:
subset = authors[authors['reviewerID'].isin(authors_sample)]
X = subset['reviewText']
Y = subset['reviewerID']
X_train = X[0:20]
Y_train = Y[0:20]
X_test = X[20:]
Y_test = Y[20:]

In [9]:
def train_and_test(X_train, Y_train, X_test, lambda_param):
    kernel = StringKernel(lambda_param=lambda_param)
    precomputed_gram_matrix = kernel.build_kernel(X_train,X_train)
    svm_instance = sklearn.svm.SVC(kernel="precomputed")
    svm_instance.fit(precomputed_gram_matrix,Y_train)

    return svm_instance.predict(kernel.build_kernel(X_test,X_test))

def get_vocab(X):
    vocab = set()
    for text in X:
        for word in nltk.word_tokenize(text):
            vocab.add(word)
    return vocab

In [ ]:
N = 10
results = []
lambdas = [0.1,0.3,0.5,0.7,0.9,0.95,1]
for i,l in enumerate(lambdas):
    print('lambda : ' + str(l))
    res = []
    for X,Y in samples:
        prediction = train_and_test(X[0:N], Y[0:N], X[N:], l)
        prec = (prediction == Y[N:]).sum() / N
        res.append(prec)
    mean = np.array(res).mean()
    print(mean)
    results.append(mean)

lambda : 0.1
0.7
lambda : 0.3
0.7
lambda : 0.5
0.7
lambda : 0.7
0.7
lambda : 0.9


In [60]:
b = np.array((3,1), dtype=str)
b[0] = 'asd'
b

array(['a', '1'],
      dtype='<U1')

In [14]:
from collections import Counter

In [78]:
kernel = StringKernel(lambda_param=1, k=1)
kernel.build_kernel(testi,testi)

array([[ 1.]])

In [53]:
testi = np.array([X.values[0][0:500]])
testi

array([ 'I find it highly interesting that Angela, the one Christian character on this show, is almost always portrayed in a negative light.  I understand that Angela is a bit judgemental of those who do not share her faith.  I know that rubs a lot of people the wrong way.  But they do not have to make her out to be a total shrew.Do I laugh at times while watching this often sleazy show?  Of course, there are many clever and laugh-out-loud moments.  I like laughing at male idiots as much as my neo-femme '],
      dtype='<U500')

In [79]:
X.shape

(20,)